##  The following notebook presents all data analysis, specially the ETL and EDA for the demographic and traffic analysis of grocery shops across the US during 2018 to 2020. The main driver to this analysis was to determine which outlet has been performing the best during the analyzed time period and the most healthier one for a possible future investiment, according to its past QoQ performance and its demographics. 

In [778]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandasql import sqldf
from datetime import date, timedelta, datetime


# Data Preparation (ETL Routines)

In [594]:
file = "c:/Users/tiago/OneDrive/Desktop/Python_Notebooks/Advent_business_case/grocers_t2_CS1.csv"

In [595]:
#Loanding the csv into a pandas df.
df=pd.read_csv(file)

In [596]:
df

,Unnamed: 0,day,devices_store,devices,Store Name,id,Address,Retailer
0,0,2015-09-10,1,360033,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
1,1,2015-09-11,1,363897,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
2,2,2015-10-09,1,499110,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
3,3,2015-10-14,1,555658,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
4,4,2015-10-15,1,570636,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
...,...,...,...,...,...,...,...,...
2167805,2167805,2020-11-12,7,11113173,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167806,2167806,2020-11-13,7,11182263,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167807,2167807,2020-11-14,20,11163652,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167808,2167808,2020-11-15,21,11093217,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market


In [597]:
# dropping duplicate columns
df.drop(['Unnamed: 0'], axis= 1,  inplace = True)
df


,day,devices_store,devices,Store Name,id,Address,Retailer
0,2015-09-10,1,360033,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
1,2015-09-11,1,363897,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
2,2015-10-09,1,499110,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
3,2015-10-14,1,555658,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
4,2015-10-15,1,570636,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
...,...,...,...,...,...,...,...
2167805,2020-11-12,7,11113173,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167806,2020-11-13,7,11182263,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167807,2020-11-14,20,11163652,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167808,2020-11-15,21,11093217,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market


In [598]:
#Renaming column to prevent while using SQL queries 
df.rename(columns={'Store Name':'Store_Name'},inplace=True)

In [599]:
#Delimiting the df to the especified timeframe (2018 to 2020). Every data previous to this point was discarded
df = df.loc[(df['day'] > '2017-12-31') & (df['day'] <= '2020-11-17')]
df

,day,devices_store,devices,Store_Name,id,Address,Retailer
509,2018-01-07,1,4491253,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
510,2018-01-10,1,4552849,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
511,2018-01-22,1,4661515,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
512,2018-01-27,1,4703098,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
513,2018-01-30,1,4722473,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC 29407",Whole Foods Market
...,...,...,...,...,...,...,...
2167805,2020-11-12,7,11113173,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167806,2020-11-13,7,11182263,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167807,2020-11-14,20,11163652,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market
2167808,2020-11-15,21,11093217,Westbury,5610153,"867 East Gate Blvd, Garden City, NY 11530",Whole Foods Market


In [600]:
# Transforming the "Address" feature and isolating the State information (letter acronyms) into a separate column ("State")
start, stop = 0, -6
df["Address"]= df["Address"].str.slice(start, stop)
df['State'] = df['Address'].str.extract(r'\b(\w+)$', expand = True)
df.head()

<ipython-input-600-674b8c317dda>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Address"]= df["Address"].str.slice(start, stop)


,day,devices_store,devices,Store_Name,id,Address,Retailer
509,2018-01-07,1,4491253,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC",Whole Foods Market
510,2018-01-10,1,4552849,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC",Whole Foods Market
511,2018-01-22,1,4661515,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC",Whole Foods Market
512,2018-01-27,1,4703098,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC",Whole Foods Market
513,2018-01-30,1,4722473,West Ashley,4653406,"1125 Savannah Hwy, Charleston, SC",Whole Foods Market


In [602]:
# Feature Transformation: Traffic normalization ("Traffic_N"), created by dividing "devices_store"(costumers pinged in a specific place) by the "devices" value (total of devices across the country).
# An arbitrary multiplication number (100000) was added just for helping with visualizations and inference further ahead, since what really matters is the proportion between both features.
df['Traffic_N'] = 100000*(df['devices_store']/df['devices'])


<ipython-input-602-11c2c0e92d9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Traffic_N'] = 100000*(df['devices_store']/df['devices'])


In [603]:
df.drop(['Address'], axis= 1,  inplace = True)
df

C:\Users\tiago\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,day,devices_store,devices,Store_Name,id,Retailer,State,Traffic_N
509,2018-01-07,1,4491253,West Ashley,4653406,Whole Foods Market,SC,0.022266
510,2018-01-10,1,4552849,West Ashley,4653406,Whole Foods Market,SC,0.021964
511,2018-01-22,1,4661515,West Ashley,4653406,Whole Foods Market,SC,0.021452
512,2018-01-27,1,4703098,West Ashley,4653406,Whole Foods Market,SC,0.021263
513,2018-01-30,1,4722473,West Ashley,4653406,Whole Foods Market,SC,0.021175
...,...,...,...,...,...,...,...,...
2167805,2020-11-12,7,11113173,Westbury,5610153,Whole Foods Market,NY,0.062988
2167806,2020-11-13,7,11182263,Westbury,5610153,Whole Foods Market,NY,0.062599
2167807,2020-11-14,20,11163652,Westbury,5610153,Whole Foods Market,NY,0.179153
2167808,2020-11-15,21,11093217,Westbury,5610153,Whole Foods Market,NY,0.189305


In [604]:
#Final check for inconsistencies
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284264 entries, 509 to 2167809
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   day            1284264 non-null  object 
 1   devices_store  1284264 non-null  int64  
 2   devices        1284264 non-null  int64  
 3   Store_Name     1284264 non-null  object 
 4   id             1284264 non-null  int64  
 5   Retailer       1284264 non-null  object 
 6   State          1284264 non-null  object 
 7   Traffic_N      1284264 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 88.2+ MB


In [ ]:
#save the pre-processed df into a new .csv file
df.to_csv('c:/Users/tiago/OneDrive/Desktop/Python_Notebooks/Advent_business_case/2018-2020-Groceries-Data-CLEAN.csv')

# Data Analysis
## Data analysis was divided and distributed across year quarters to facilitate the analysis and to help provide meaningful insights on the granularity of the variation between each year and each fiscar quarter. 

# --------------------------------------------------2018 ----------------------------------------------------------------------------------

## 1st Quarter 2018

In [666]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q18_1= df.loc[(df['day'] >= '2018-01-01') & (df['day'] <= '2018-03-31')]

Traffic_Q18_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State
    FROM Q18_1
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q18_1.head()


,Store_Name,Retailer,Traffic_N,State
0,Richard Cooklin,Freshmarket,2.251014,VA
1,Fairview,Whole Foods Market,2.113197,TX
2,Clearwater,Whole Foods Market,2.067258,FL
3,Cupertino,Whole Foods Market,1.538958,CA
4,Midtown East (E 57th St),Whole Foods Market,1.529039,NY


In [606]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q18_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id,
    SUM (Traffic_N)
    FROM Q18_1
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q18_1

,Store_Name,Retailer,Traffic_N,State,id,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.022266,SC,4653406,22591.673657
1,Tulsa (Store #805),Sprouts Farmers Market,0.137817,OK,5610149,11580.168251
2,Morgan Mislan,Freshmarket,0.183756,FL,2408475,2991.241805
3,Mt. Prospect (Store #101),Freshthyme,0.137817,IL,5610126,1851.880144
4,New Mexico,naturalgrocer,0.045939,NM,5610145,1638.290765
5,"Asheville, NC",EARTHFARE,0.183756,NC,5610056,299.371433


In [743]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q18_1= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q18_1
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q18_1

,COUNT(Store_Name),Retailer,State
0,10441,Sprouts Farmers Market,CA
1,7811,Whole Foods Market,CA
2,3950,Sprouts Farmers Market,TX
3,3940,Freshmarket,FL
4,3339,Sprouts Farmers Market,AZ
...,...,...,...
119,43,naturalgrocer,LA
120,38,Sprouts Farmers Market,SC
121,25,Sprouts Farmers Market,VA
122,24,Sprouts Farmers Market,LA


## 2nd Quarter 2018

In [608]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q18_2= df.loc[(df['day'] >= '2018-04-01') & (df['day'] <= '2018-06-30')]

Traffic_Q18_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_2
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)

Traffic_Q18_2

,Store_Name,Retailer,Traffic_N,State,id
0,Cupertino,Whole Foods Market,2.019386,CA,4231451
1,Fairview,Whole Foods Market,1.942457,TX,4231285
2,Plano,Whole Foods Market,1.903992,TX,4231396
3,Lamar,Whole Foods Market,1.769367,TX,4231339
4,Naples,Whole Foods Market,1.634741,FL,4231371


In [609]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q18_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q18_2
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q18_2.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.038290,SC,23049.688519
1,Tulsa (Store #805),Sprouts Farmers Market,0.269251,OK,11567.913944
2,Morgan Mislan,Freshmarket,0.326948,FL,3161.283719
3,Mt. Prospect (Store #101),Freshthyme,0.096161,IL,1856.546023
4,New Mexico,naturalgrocer,0.038464,NM,1566.451349
5,"Asheville, NC",EARTHFARE,0.346180,NC,280.253839


In [744]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q18_2= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q18_2
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q18_2

,COUNT(Store_Name),Retailer,State
0,10706,Sprouts Farmers Market,CA
1,8019,Whole Foods Market,CA
2,4181,Freshmarket,FL
3,4021,Sprouts Farmers Market,TX
4,3447,Sprouts Farmers Market,AZ
...,...,...,...
119,86,Sprouts Farmers Market,SC
120,83,naturalgrocer,LA
121,44,Sprouts Farmers Market,VA
122,27,Sprouts Farmers Market,LA


## 3rd Quarter 2018 

In [610]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q18_3= df.loc[(df['day'] >= '2018-07-01') & (df['day'] <= '2018-09-30')]
Traffic_Q18_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, id
    FROM Q18_3
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q18_3

,Store_Name,Retailer,Traffic_N,State,id
0,Richard Cooklin,Freshmarket,2.788678,VA,3838593
1,Queen,Whole Foods Market,2.737352,HI,4973613
2,Clearwater,Whole Foods Market,1.830604,FL,4231581
3,Avalon,Whole Foods Market,1.796388,GA,4231578
4,Dublin Ca,Whole Foods Market,1.693737,CA,4231604


In [611]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q18_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q18_3
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q18_3.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.034217,SC,20992.272836
1,Tulsa (Store #805),Sprouts Farmers Market,0.325061,OK,9468.411934
2,Morgan Mislan,Freshmarket,0.393494,FL,2753.854646
3,Mt. Prospect (Store #101),Freshthyme,0.290844,IL,1688.334926
4,New Mexico,naturalgrocer,0.051325,NM,1529.330345
5,"Asheville, NC",EARTHFARE,0.273735,NC,279.984809


In [746]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q18_3= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q18_3
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q18_3

,COUNT(Store_Name),Retailer,State
0,10916,Sprouts Farmers Market,CA
1,8185,Whole Foods Market,CA
2,4220,Freshmarket,FL
3,4104,Sprouts Farmers Market,TX
4,3506,Sprouts Farmers Market,AZ
...,...,...,...
119,88,Freshmarket,OK
120,67,Freshmarket,DE
121,51,Sprouts Farmers Market,PA
122,27,Sprouts Farmers Market,VA


## 4th Quarter 2018

In [612]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q18_4= df.loc[(df['day'] >= '2018-10-01') & (df['day'] <= '2018-12-31')]
Traffic_Q18_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_4
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q18_4

,Store_Name,Retailer,Traffic_N,State,id
0,Coral Gables,Whole Foods Market,2.485408,FL,4231235
1,Richard Cooklin,Freshmarket,2.054412,VA,3838593
2,Downtown Los Angeles,Whole Foods Market,1.522851,CA,4231615
3,Estero (Store #631),Sprouts Farmers Market,1.422285,FL,5586491
4,Davie,Whole Foods Market,1.407919,FL,4231394


In [613]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q18_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q18_4
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q18_4.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.028733,SC,20502.249672
1,Tulsa (Store #805),Sprouts Farmers Market,0.416629,OK,8502.425723
2,Morgan Mislan,Freshmarket,0.545928,FL,2914.207140
3,Mt. Prospect (Store #101),Freshthyme,0.201131,IL,1634.500497
4,New Mexico,naturalgrocer,0.014367,NM,1555.794171
5,"Asheville, NC",EARTHFARE,0.316063,NC,257.824058


In [747]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q18_4= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q18_4
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q18_4

,COUNT(Store_Name),Retailer,State
0,11061,Sprouts Farmers Market,CA
1,8196,Whole Foods Market,CA
2,4187,Freshmarket,FL
3,4142,Sprouts Farmers Market,TX
4,3554,naturalgrocer,CO
...,...,...,...
119,88,Freshmarket,OK
120,86,Sprouts Farmers Market,N
121,71,Freshmarket,DE
122,48,Sprouts Farmers Market,LA


# --------------------------------------------------2019----------------------------------------------------------------------------------

## 1st Quarter 2019

In [614]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q19_1= df.loc[(df['day'] >= '2019-01-01') & (df['day'] <= '2019-03-31')]

Traffic_Q19_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_1
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)

Traffic_Q19_1.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Richard Cooklin,Freshmarket,2.043223,VA,3838593
1,Queen,Whole Foods Market,1.587727,HI,4973613
2,Fairview,Whole Foods Market,1.587727,TX,4231285
3,Orlando,Whole Foods Market,1.314430,FL,4231210
4,Annapolis,Whole Foods Market,1.249359,MD,4231506


In [615]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q19_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q19_1
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q19_1.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.013014,SC,19521.177678
1,Tulsa (Store #805),Sprouts Farmers Market,0.377411,OK,8499.838988
2,Morgan Mislan,Freshmarket,0.351382,FL,2484.152404
3,Mt. Prospect (Store #101),Freshthyme,0.143156,IL,1764.364786
4,New Mexico,naturalgrocer,0.078085,NM,1503.842580
5,"Asheville, NC",EARTHFARE,0.156170,NC,263.840109


In [748]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q19_1= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q19_1
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q19_1

,COUNT(Store_Name),Retailer,State
0,10955,Sprouts Farmers Market,CA
1,8058,Whole Foods Market,CA
2,4132,Freshmarket,FL
3,4106,Sprouts Farmers Market,TX
4,3546,naturalgrocer,CO
...,...,...,...
119,85,Freshmarket,OK
120,75,Sprouts Farmers Market,N
121,66,Freshmarket,DE
122,63,Sprouts Farmers Market,LA


### Calculating QoQ growth (1Q18 vs 1Q19)

In [616]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key) 
SortbyID_Q19_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_1
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q18_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_1
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)


In [617]:
#Calculation variation in traffic ("Growth", in %)
GROW_Q19_1 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q19_1
    INNER JOIN SortbyID_Q18_1 ON [SortbyID_Q19_1].id=SortbyID_Q18_1.id 
"""
)
GROW_Q19_1.tail()

,Store_Name,Retailer,Traffic_N,State,id,Store_Name,Retailer,Traffic_N,State,id
1258,Texas,naturalgrocer,0.167980,TX,766396,Texas,naturalgrocer,0.136171,TX,766396
1259,Texas,naturalgrocer,0.038765,TX,766394,Texas,naturalgrocer,0.045390,TX,766394
1260,Colorado,naturalgrocer,0.026028,CO,766393,Colorado,naturalgrocer,0.158866,CO,766393
1261,Texas,naturalgrocer,0.078085,TX,766392,Texas,naturalgrocer,0.137817,TX,766392
1262,Colorado,naturalgrocer,0.038765,CO,766391,Colorado,naturalgrocer,0.022970,CO,766391


In [618]:
#dropping duplicates
GROW_Q19_1.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_18', 'Retailer_18', 'Traffic_N_18','State_18','id_18']
GROW_Q19_1.drop(['Store_Name_18','Retailer_18', 'State_18', 'id_18'], axis=1, inplace=True)

In [619]:
#RCalculating traffic growth data
GROW_Q19_1['Growth'] = 100*((GROW_Q19_1['Traffic_N']/GROW_Q19_1['Traffic_N_18'])-1).mask(GROW_Q19_1.isna(), np.nan)
GROW_Q19_1

,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_18,Growth
0,Westbury,Whole Foods Market,0.025843,NY,5610153,0.022970,12.509996
1,17th Street,Whole Foods Market,0.026028,FL,5610152,0.091878,-71.670823
2,Sunnyvale,Whole Foods Market,0.026028,CA,5610151,0.022970,13.316710
3,Sherman Oaks,Whole Foods Market,0.091099,CA,5610150,0.137817,-33.898586
4,Tulsa (Store #805),Sprouts Farmers Market,0.377411,OK,5610149,0.137817,173.848716
...,...,...,...,...,...,...,...
1258,Texas,naturalgrocer,0.167980,TX,766396,0.136171,23.359738
1259,Texas,naturalgrocer,0.038765,TX,766394,0.045390,-14.597104
1260,Colorado,naturalgrocer,0.026028,CO,766393,0.158866,-83.616143
1261,Texas,naturalgrocer,0.078085,TX,766392,0.137817,-43.341645


In [620]:
#Ranking QoQ Traffic growth (in %) data
GROW_Q19_1.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q19_1

,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_18,Growth
168,Chappaqua,Whole Foods Market,1.028118,NY,4973614,0.022040,4564.754071
264,Orange Village,Whole Foods Market,0.650708,OH,4653413,0.022500,2792.074248
212,Mira Mesa (Store #430),Sprouts Farmers Market,0.624680,CA,4680473,0.022970,2619.601038
266,University Station,Whole Foods Market,0.494538,FL,4653410,0.022970,2053.017488
274,Exton,Whole Foods Market,0.403439,PA,4653402,0.022970,1656.409004
...,...,...,...,...,...,...,...
238,Colorado,naturalgrocer,0.013014,CO,4677072,0.181561,-92.832063
739,Iowa,naturalgrocer,0.013014,IA,2548848,0.204256,-93.628500
969,Albuquerque - Four Hills (Store #707),Sprouts Farmers Market,0.013014,NM,889285,0.206726,-93.704627
160,Jacksonville (Store #624),Sprouts Farmers Market,0.013014,FL,5030430,0.275634,-95.278470


In [621]:
#Ranking growth data by retailer 
RETAILER_GROW_Q19_1 = sqldf(
"""
    SELECT Retailer, Growth,
    SUM (Growth) 
    FROM GROW_Q19_1
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q19_1.head(6)


,Retailer,Growth,SUM (Growth)
0,Whole Foods Market,4564.754071,22688.526336
1,Sprouts Farmers Market,2619.601038,15056.487693
2,Freshthyme,1562.961449,9570.678707
3,Freshmarket,551.571082,4910.995189
4,naturalgrocer,531.726759,1878.666974
5,EARTHFARE,372.380159,839.080101


## 2nd Quarter 2019

In [622]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q19_2= df.loc[(df['day'] >= '2019-04-01') & (df['day'] <= '2019-06-30')]

Traffic_Q19_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q19_2
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)

Traffic_Q19_2.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Coral Gables,Whole Foods Market,2.117024,FL,4231235
1,Richard Cooklin,Freshmarket,1.874086,VA,3838593
2,Estero (Store #631),Sprouts Farmers Market,1.827813,FL,5586491
3,Midtown East (E 57th St),Whole Foods Market,1.503896,NY,4231349
4,Avalon,Whole Foods Market,1.422917,GA,4231578


In [623]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q19_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q19_2
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)
Best_Retailer_Q19_2.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.046274,SC,20389.423219
1,Tulsa (Store #805),Sprouts Farmers Market,0.196663,OK,8797.852384
2,Morgan Mislan,Freshmarket,0.358621,FL,2549.343399
3,Mt. Prospect (Store #101),Freshthyme,0.127253,IL,1779.322145
4,New Mexico,naturalgrocer,0.034705,NM,1467.884054
5,"Asheville, NC",EARTHFARE,0.323916,NC,254.810117


In [749]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q19_2= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q19_2
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q19_2

,COUNT(Store_Name),Retailer,State
0,11158,Sprouts Farmers Market,CA
1,8188,Whole Foods Market,CA
2,4182,Freshmarket,FL
3,4172,Sprouts Farmers Market,TX
4,3614,Sprouts Farmers Market,AZ
...,...,...,...
119,88,Sprouts Farmers Market,N
120,84,Sprouts Farmers Market,LA
121,78,naturalgrocer,LA
122,76,Freshmarket,DE


### Calculating QoQ growth (1Q18 vs 1Q19)

In [624]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q19_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_2
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q18_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_2
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q19_2 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q19_2
    INNER JOIN SortbyID_Q18_2 ON [SortbyID_Q19_2].id=SortbyID_Q18_2.id 
"""
)

GROW_Q19_2.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_18', 'Retailer_18', 'Traffic_N_18','State_18','id_18']
GROW_Q19_2.drop(['Store_Name_18','Retailer_18', 'State_18', 'id_18'], axis=1, inplace=True)
GROW_Q19_2['Growth'] = 100*((GROW_Q19_2['Traffic_N']/GROW_Q19_2['Traffic_N_18'])-1).mask(GROW_Q19_2.isna(), np.nan)
GROW_Q19_2
GROW_Q19_2.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q19_2



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_18,Growth
211,Mira Mesa (Store #430),Sprouts Farmers Market,0.682538,CA,4680473,0.019232,3448.923361
167,Chappaqua,Whole Foods Market,0.601559,NY,4973614,0.019036,3060.052854
170,Newtown,Whole Foods Market,0.566853,PA,4973610,0.019232,2847.410927
91,Cranberry Township (Store #453),Freshthyme,0.485874,PA,5610062,0.019132,2439.648657
182,Sugar Land (Store #152),Sprouts Farmers Market,0.404895,TX,4775070,0.019079,2022.244052
...,...,...,...,...,...,...,...
110,Houston (Store #156),Sprouts Farmers Market,0.011568,TX,5586495,0.153858,-92.481095
138,Pasadena (Store #919),Sprouts Farmers Market,0.011406,MD,5156150,0.153858,-92.586859
113,Seattle (Store #457),Sprouts Farmers Market,0.011568,WA,5586492,0.173090,-93.316529
145,West Covina (Store #438),Sprouts Farmers Market,0.011568,CA,5030445,0.519271,-97.772176


In [625]:
#Ranking growth data by retailer 
RETAILER_GROW_Q19_2 = sqldf(
"""
    SELECT Retailer, Growth,
    SUM (Growth)  
    FROM GROW_Q19_2
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q19_2.head(6)

,Retailer,Growth,SUM (Growth)
0,Sprouts Farmers Market,3448.923361,21707.034237
1,Whole Foods Market,3060.052854,14957.870434
2,Freshthyme,2439.648657,15248.093447
3,Freshmarket,1163.176111,3675.160219
4,naturalgrocer,802.268651,19173.936977
5,EARTHFARE,381.209947,1281.004792


## 3rd Quarter 2019

In [626]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q19_3= df.loc[(df['day'] >= '2019-07-01') & (df['day'] <= '2019-09-30')]
Traffic_Q19_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q19_3
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q19_3.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Union Square,Whole Foods Market,3.202754,NY,4231472
1,Richard Cooklin,Freshmarket,2.446694,VA,3838593
2,Coral Gables,Whole Foods Market,1.942654,FL,4231235
3,Avalon,Whole Foods Market,1.575125,GA,4231578
4,Estero (Store #631),Sprouts Farmers Market,1.554123,FL,5586491


In [627]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quart
Best_Retailer_Q19_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q19_3
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q19_3.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.084007,SC,19931.456506
1,Tulsa (Store #805),Sprouts Farmers Market,0.241519,OK,9024.416012
2,Morgan Mislan,Freshmarket,0.294023,FL,2524.382112
3,Mt. Prospect (Store #101),Freshthyme,0.157512,IL,1769.259709
4,New Mexico,naturalgrocer,0.063005,NM,1527.668713
5,"Asheville, NC",EARTHFARE,0.283522,NC,256.411118


In [750]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q19_3= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q19_3
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q19_3

,COUNT(Store_Name),Retailer,State
0,11370,Sprouts Farmers Market,CA
1,8390,Whole Foods Market,CA
2,4234,Sprouts Farmers Market,TX
3,4222,Freshmarket,FL
4,3717,Sprouts Farmers Market,AZ
...,...,...,...
119,92,Sprouts Farmers Market,SC
120,90,Freshmarket,DE
121,79,Sprouts Farmers Market,VA
122,77,naturalgrocer,LA


### Calculating QoQ growth (3Q18 vs 3Q19)

In [628]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q19_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_3
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q18_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_3
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q19_3 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q19_3
    INNER JOIN SortbyID_Q18_3 ON [SortbyID_Q19_3].id=SortbyID_Q18_3.id 
"""
)

GROW_Q19_3.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_18', 'Retailer_18', 'Traffic_N_18','State_18','id_18']
GROW_Q19_3.drop(['Store_Name_18','Retailer_18', 'State_18', 'id_18'], axis=1, inplace=True)
GROW_Q19_3['Growth'] = 100*((GROW_Q19_3['Traffic_N']/GROW_Q19_3['Traffic_N_18'])-1).mask(GROW_Q19_3.isna(), np.nan)
GROW_Q19_3
GROW_Q19_3.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q19_3



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_18,Growth
175,Porter Ranch,Whole Foods Market,0.808564,CA,4973606,0.017049,4642.533032
153,San Antonio - Bulverde Rd. (Store #123),Sprouts Farmers Market,0.682554,TX,5030438,0.016599,4012.016312
145,Mesa-Signal Butte (Store #43),Sprouts Farmers Market,0.609048,AZ,5030446,0.015948,3718.974331
158,Baton Rouge (Store #691),Sprouts Farmers Market,0.441035,LA,5030433,0.016982,2497.042189
171,Newtown,Whole Foods Market,0.441035,PA,4973610,0.017108,2477.877456
...,...,...,...,...,...,...,...
111,Miramar (Store #626),Sprouts Farmers Market,0.010501,FL,5586494,0.102295,-89.734777
1236,Utah,naturalgrocer,0.010515,UT,766428,0.102651,-89.756598
1232,Texas,naturalgrocer,0.010501,TX,766432,0.102651,-89.770328
947,Donald Conner,Freshmarket,0.010501,NC,911347,0.205301,-94.885164


In [629]:
#Ranking growth data by retailer 
RETAILER_GROW_Q19_3 = sqldf(
"""
    SELECT Retailer, Growth,
    SUM (Growth) 
    FROM GROW_Q19_3
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q19_3.head(6)

,Retailer,Growth,SUM (Growth)
0,Whole Foods Market,4642.533032,18293.814868
1,Sprouts Farmers Market,4012.016312,24164.218477
2,Freshthyme,2055.696833,5611.745616
3,naturalgrocer,575.158381,4914.925904
4,Freshmarket,268.268208,-1168.719733
5,EARTHFARE,76.461850,-129.561185


## 4th Quarter 2019

In [630]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q19_4= df.loc[(df['day'] >= '2019-10-01') & (df['day'] <= '2019-12-31')]
Traffic_Q19_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q19_4
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q19_4.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Coral Gables,Whole Foods Market,1.791325,FL,4231235
1,Richard Cooklin,Freshmarket,1.625120,VA,3838593
2,Estero (Store #631),Sprouts Farmers Market,1.588185,FL,5586491
3,Boca Raton,Whole Foods Market,1.394279,FL,4231204
4,Hyde Park,Whole Foods Market,1.320410,IL,4231646


In [631]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q19_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q19_4
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q19_4.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.092336,SC,18488.325652
1,Tulsa (Store #805),Sprouts Farmers Market,0.258542,OK,7710.772766
2,Morgan Mislan,Freshmarket,0.443214,FL,2539.453650
3,Mt. Prospect (Store #101),Freshthyme,0.129271,IL,1553.614741
4,New Mexico,naturalgrocer,0.120037,NM,1442.883800
5,"Asheville, NC",EARTHFARE,0.221607,NC,222.209785


In [766]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q19_4= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q19_4
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q19_4

,COUNT(Store_Name),Retailer,State
0,11315,Sprouts Farmers Market,CA
1,8374,Whole Foods Market,CA
2,4224,Sprouts Farmers Market,TX
3,4204,Freshmarket,FL
4,3756,Sprouts Farmers Market,AZ
...,...,...,...
119,91,Sprouts Farmers Market,N
120,91,Sprouts Farmers Market,VA
121,90,naturalgrocer,LA
122,90,Freshmarket,M


### Calculating QoQ growth (4Q18 vs 4Q19)

In [632]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q19_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_4
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q18_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q18_4
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q19_4 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q19_4
    INNER JOIN SortbyID_Q18_4 ON [SortbyID_Q19_4].id=SortbyID_Q18_4.id 
"""
)

GROW_Q19_4.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_18', 'Retailer_18', 'Traffic_N_18','State_18','id_18']
GROW_Q19_4.drop(['Store_Name_18','Retailer_18', 'State_18', 'id_18'], axis=1, inplace=True)
GROW_Q19_4['Growth'] = 100*((GROW_Q19_4['Traffic_N']/GROW_Q19_4['Traffic_N_18'])-1).mask(GROW_Q19_4.isna(), np.nan)
GROW_Q19_4
GROW_Q19_4.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q19_4



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_18,Growth
176,Porter Ranch,Whole Foods Market,0.544784,CA,4973606,0.014332,3701.277409
145,Mesa-Signal Butte (Store #43),Sprouts Farmers Market,0.489383,AZ,5030446,0.013959,3405.773099
153,San Antonio - Bulverde Rd. (Store #123),Sprouts Farmers Market,0.433981,TX,5030438,0.014332,2928.136241
149,Trinity (Store #622),Sprouts Farmers Market,0.397046,FL,5030442,0.014367,2663.691790
175,Commack,Whole Foods Market,0.941831,NY,4973607,0.043100,2085.244671
...,...,...,...,...,...,...,...
12,Brookhurst,Whole Foods Market,0.009234,CA,5610141,0.071833,-87.145620
1191,Kansas,naturalgrocer,0.009234,KS,766475,0.071833,-87.145620
26,Florida Avenue,Whole Foods Market,0.009234,DC,5610127,0.086199,-89.288016
948,Donald Conner,Freshmarket,0.009234,NC,911347,0.086199,-89.288016


In [633]:
#Ranking growth data by retailer 
RETAILER_GROW_Q19_4 = sqldf(
"""
    SELECT Retailer, Growth ,
    SUM (Growth) 
    FROM GROW_Q19_4
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q19_4.head(6)

,Retailer,Growth,SUM (Growth)
0,Whole Foods Market,3701.277409,10423.936995
1,Sprouts Farmers Market,3405.773099,23928.883859
2,Freshthyme,1058.054351,3960.002212
3,Freshmarket,928.350433,7337.863544
4,naturalgrocer,864.078531,3317.689239
5,EARTHFARE,6.188360,-291.651620


# --------------------------------------------------2020 ----------------------------------------------------------------------------------

## 1st Quarter 2020

In [634]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q20_1= df.loc[(df['day'] >= '2020-01-01') & (df['day'] <= '2020-03-31')]

Traffic_Q20_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q20_1
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)

Traffic_Q20_1.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Richard Cooklin,Freshmarket,1.954717,VA,3838593
1,Queen,Whole Foods Market,1.477298,HI,4973613
2,Estero (Store #631),Sprouts Farmers Market,1.405234,FL,5586491
3,Fairview,Whole Foods Market,1.306147,TX,4231285
4,Lamar,Whole Foods Market,1.216068,TX,4231339


In [635]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q20_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q20_1
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q20_1.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.054047,SC,16925.751575
1,Tulsa (Store #805),Sprouts Farmers Market,0.180158,OK,8135.137886
2,Morgan Mislan,Freshmarket,0.180158,FL,2086.381595
3,Mt. Prospect (Store #101),Freshthyme,0.153135,IL,1576.920937
4,New Mexico,naturalgrocer,0.027024,NM,1419.673311
5,"Asheville, NC",EARTHFARE,0.090079,NC,151.782253


In [753]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q20_1= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q20_1
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q20_1

,COUNT(Store_Name),Retailer,State
0,11263,Sprouts Farmers Market,CA
1,8320,Whole Foods Market,CA
2,4201,Sprouts Farmers Market,TX
3,4173,Freshmarket,FL
4,3768,Sprouts Farmers Market,AZ
...,...,...,...
119,89,naturalgrocer,MN
120,89,EARTHFARE,S
121,88,EARTHFARE,GA
122,79,EARTHFARE,MI


### Calculating QoQ growth (1Q19 vs 1Q20)

In [636]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q20_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q20_1
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q19_1 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_1
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q20_1 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q20_1
    INNER JOIN SortbyID_Q19_1 ON [SortbyID_Q20_1].id=SortbyID_Q19_1.id 
"""
)

GROW_Q20_1.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_19', 'Retailer_19', 'Traffic_N_19','State_19','id_19']
GROW_Q20_1.drop(['Store_Name_19','Retailer_19', 'State_19', 'id_19'], axis=1, inplace=True)
GROW_Q20_1['Growth'] = 100*((GROW_Q20_1['Traffic_N']/GROW_Q20_1['Traffic_N_19'])-1).mask(GROW_Q20_1.isna(), np.nan)
GROW_Q20_1
GROW_Q20_1.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q20_1



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_19,Growth
176,Porter Ranch,Whole Foods Market,0.639562,CA,4973606,0.012922,4849.590689
149,Trinity (Store #622),Sprouts Farmers Market,0.468411,FL,5030442,0.013014,3499.245209
145,Mesa-Signal Butte (Store #43),Sprouts Farmers Market,0.423372,AZ,5030446,0.012433,3305.353843
124,Beverly,Whole Foods Market,0.360316,M,5181993,0.012796,2715.847295
153,San Antonio - Bulverde Rd. (Store #123),Sprouts Farmers Market,0.315277,TX,5030438,0.012817,2359.850386
...,...,...,...,...,...,...,...
822,Mike Paholik,Freshmarket,0.009008,FL,911512,0.091099,-90.111964
237,Texas,naturalgrocer,0.009008,TX,4677074,0.103372,-91.285932
1225,Kansas,naturalgrocer,0.009008,KS,766440,0.103372,-91.285932
939,Jonathan Hall,Freshmarket,0.018016,NC,911357,0.208227,-91.347968


In [637]:
#Ranking growth data by retailer 
RETAILER_GROW_Q20_1 = sqldf(
"""
    SELECT Retailer, Growth ,
    SUM (Growth) 
    FROM GROW_Q20_1
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q20_1.head(6)

,Retailer,Growth,SUM (Growth)
0,Whole Foods Market,4849.590689,10707.410621
1,Sprouts Farmers Market,3499.245209,26092.107284
2,Freshthyme,945.688174,2809.946183
3,naturalgrocer,862.540649,5589.688370
4,Freshmarket,384.513778,-2816.345694
5,EARTHFARE,69.894442,-76.207522


## 2nd Quarter 2020

In [638]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q20_2= df.loc[(df['day'] >= '2020-04-01') & (df['day'] <= '2020-06-30')]

Traffic_Q20_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q20_2
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)

Traffic_Q20_2.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Estero (Store #631),Sprouts Farmers Market,0.941377,FL,5586491
1,Fairview,Whole Foods Market,0.843687,TX,4231285
2,Avalon,Whole Foods Market,0.639426,GA,4231578
3,Coral Gables,Whole Foods Market,0.577259,FL,4231235
4,Dublin Ca,Whole Foods Market,0.550617,CA,4231604


In [639]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q20_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q20_2
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q20_2.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.062166,SC,9350.214831
1,Tulsa (Store #805),Sprouts Farmers Market,0.159856,OK,7092.099881
2,Morgan Mislan,Freshmarket,0.222023,FL,1641.518855
3,Mt. Prospect (Store #101),Freshthyme,0.079928,IL,1253.587349
4,New Mexico,naturalgrocer,0.026643,NM,1145.526803
5,"Asheville, NC",EARTHFARE,0.009101,NC,22.645097


In [754]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q20_2= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q20_2
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q20_2

,COUNT(Store_Name),Retailer,State
0,11146,Sprouts Farmers Market,CA
1,8264,Whole Foods Market,CA
2,4187,Sprouts Farmers Market,TX
3,4146,Freshmarket,FL
4,3731,Sprouts Farmers Market,AZ
...,...,...,...
119,90,Sprouts Farmers Market,VA
120,88,naturalgrocer,MN
121,77,EARTHFARE,GA
122,74,EARTHFARE,S


### Calculating QoQ growth (2Q19 vs 2Q20)

In [640]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q20_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q20_2
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q19_2 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_2
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q20_2 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q20_2
    INNER JOIN SortbyID_Q19_2 ON [SortbyID_Q20_2].id=SortbyID_Q19_2.id 
"""
)

GROW_Q20_2.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_19', 'Retailer_19', 'Traffic_N_19','State_19','id_19']
GROW_Q20_2.drop(['Store_Name_19','Retailer_19', 'State_19', 'id_19'], axis=1, inplace=True)
GROW_Q20_2['Growth'] = 100*((GROW_Q20_2['Traffic_N']/GROW_Q20_2['Traffic_N_19'])-1).mask(GROW_Q20_2.isna(), np.nan)
GROW_Q20_2
GROW_Q20_2.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q20_2



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_19,Growth
145,West Covina (Store #438),Sprouts Farmers Market,0.248666,CA,5030445,0.011568,2049.517576
153,San Jose - Capitol Expwy. (Store #433),Sprouts Farmers Market,0.230904,CA,5030437,0.011530,1902.617933
158,Jupiter (Store #625),Sprouts Farmers Market,0.230904,FL,5030432,0.011568,1895.980606
139,Cave Creek (Store #44),Sprouts Farmers Market,0.213142,AZ,5156149,0.011557,1744.235946
152,San Antonio - Bulverde Rd. (Store #123),Sprouts Farmers Market,0.390760,TX,5030438,0.023114,1590.549617
...,...,...,...,...,...,...,...
303,Bryant Park,Whole Foods Market,0.026643,NY,4231643,0.694106,-96.161576
822,Troy Hendrix,Freshmarket,0.009010,FL,911511,0.254506,-96.459965
108,"Roanoke, VA",EARTHFARE,0.008881,VA,5610008,0.266074,-96.662240
97,"Asheville, NC",EARTHFARE,0.009101,NC,5610056,0.323916,-97.190435


In [641]:
#Ranking growth data by retailer 
RETAILER_GROW_Q20_2 = sqldf(
"""
    SELECT Retailer, Growth,
    SUM (Growth)  
    FROM GROW_Q20_2
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q20_2.head(6)

,Retailer,Growth,SUM (Growth)
0,Sprouts Farmers Market,2049.517576,12892.535747
1,Whole Foods Market,1205.064242,-21760.775406
2,Freshthyme,1128.295757,-432.694835
3,Freshmarket,668.431644,-1491.370704
4,naturalgrocer,283.842424,-3950.095105
5,EARTHFARE,-79.528404,-1148.344629


## 3rd Quarter 2020

In [642]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q20_3= df.loc[(df['day'] >= '2020-07-01') & (df['day'] <= '2020-09-30')]
Traffic_Q20_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q20_3
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q20_3.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Richard Cooklin,Freshmarket,1.787366,VA,3838593
1,Estero (Store #631),Sprouts Farmers Market,1.391197,FL,5586491
2,Coral Gables,Whole Foods Market,1.050308,FL,4231235
3,Fairview,Whole Foods Market,1.004242,TX,4231285
4,Utah,naturalgrocer,0.875257,UT,5175333


In [643]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q20_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q20_3
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q20_3.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.055279,SC,11513.933340
1,Tulsa (Store #805),Sprouts Farmers Market,0.340889,OK,8311.436340
2,Morgan Mislan,Freshmarket,0.147412,FL,1837.899841
3,Mt. Prospect (Store #101),Freshthyme,0.138198,IL,1339.670403
4,New Mexico,naturalgrocer,0.036853,NM,1277.669418
5,"Asheville, NC",EARTHFARE,0.119772,NC,97.341535


In [755]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q20_3= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q20_3
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q20_3

,COUNT(Store_Name),Retailer,State
0,11361,Sprouts Farmers Market,CA
1,8400,Whole Foods Market,CA
2,4285,Sprouts Farmers Market,TX
3,4227,Freshmarket,FL
4,3804,Sprouts Farmers Market,AZ
...,...,...,...
119,91,EARTHFARE,GA
120,91,naturalgrocer,LA
121,91,naturalgrocer,MN
122,88,EARTHFARE,S


### Calculating QoQ growth (3Q19 vs 3Q20)

In [644]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q20_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q20_3
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q19_3 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_3
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q20_3 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q20_3
    INNER JOIN SortbyID_Q19_3 ON [SortbyID_Q20_3].id=SortbyID_Q19_3.id 
"""
)

GROW_Q20_3.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_19', 'Retailer_19', 'Traffic_N_19','State_19','id_19']
GROW_Q20_3.drop(['Store_Name_19','Retailer_19', 'State_19', 'id_19'], axis=1, inplace=True)
GROW_Q20_3['Growth'] = 100*((GROW_Q20_3['Traffic_N']/GROW_Q20_3['Traffic_N_19'])-1).mask(GROW_Q20_3.isna(), np.nan)
GROW_Q20_3
GROW_Q20_3.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q20_3



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_19,Growth
140,Cave Creek (Store #44),Sprouts Farmers Market,0.396169,AZ,5156149,0.010515,3667.680303
139,Pasadena (Store #919),Sprouts Farmers Market,0.267184,MD,5156150,0.010515,2440.993693
156,Greensboro (Store #544),Sprouts Farmers Market,0.248757,NC,5030435,0.010490,2271.314958
21,Castle Rock,Whole Foods Market,0.497514,CO,5610132,0.021030,2265.752748
111,Miramar (Store #626),Sprouts Farmers Market,0.211904,FL,5586494,0.010501,1917.975605
...,...,...,...,...,...,...,...
652,Columbus Circle,Whole Foods Market,0.009204,NY,4231246,0.168013,-94.521797
109,"Williamsburg, VA",EARTHFARE,0.009248,VA,5610007,0.178514,-94.819421
100,"Charleston, SC",EARTHFARE,0.009213,SC,5610043,0.178514,-94.838937
1084,Redondo Beach (Store #222),Sprouts Farmers Market,0.009209,CA,889165,0.178514,-94.841502


In [645]:
#Ranking growth data by retailer 
RETAILER_GROW_Q20_3 = sqldf(
"""
    SELECT Retailer, Growth ,
    SUM (Growth) 
    FROM GROW_Q20_3
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q20_3.head(6)

,Retailer,Growth,SUM (Growth)
0,Sprouts Farmers Market,3667.680303,23861.492355
1,Whole Foods Market,2265.752748,-15390.644554
2,Freshthyme,1652.409443,1009.704178
3,Freshmarket,1479.285256,-425.834651
4,naturalgrocer,689.642628,1593.476507
5,EARTHFARE,33.696106,-972.835119


## 4th Quarter 2020

In [646]:
#Providing the traffic data aggregation for each retailer and store, ordered by the traffic measured in each store.
Q20_4= df.loc[(df['day'] >= '2020-10-01') & (df['day'] <= '2020-12-31')]
Traffic_Q20_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State ,id
    FROM Q20_4
    GROUP BY (Store_Name)
    ORDER BY (Traffic_N)
    DESC

"""
)
Traffic_Q20_4.head()

,Store_Name,Retailer,Traffic_N,State,id
0,Estero (Store #631),Sprouts Farmers Market,1.576090,FL,5586491
1,Richard Cooklin,Freshmarket,1.520464,VA,3838593
2,Coral Gables,Whole Foods Market,1.093992,FL,4231235
3,Utah,naturalgrocer,0.806587,UT,5175333
4,Avalon,Whole Foods Market,0.788045,GA,4231578


In [647]:
#Ranking the best and worst performing stores, in regard to absolute traffic during the Quarter
Best_Retailer_Q20_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State, 
    SUM (Traffic_N)
    FROM Q20_4
    GROUP BY (Retailer)
    ORDER BY (SUM(Traffic_N))
    DESC
"""
)

Best_Retailer_Q20_4.head(6)

,Store_Name,Retailer,Traffic_N,State,SUM (Traffic_N)
0,West Ashley,Whole Foods Market,0.037084,SC,5960.303239
1,Tulsa (Store #805),Sprouts Farmers Market,0.185422,OK,3957.742487
2,Morgan Mislan,Freshmarket,0.166880,FL,959.504400
3,New Mexico,naturalgrocer,0.176151,NM,650.088253
4,Mt. Prospect (Store #101),Freshthyme,0.074169,IL,634.928510
5,"Asheville, NC",EARTHFARE,0.139067,NC,69.357197


In [765]:
#Defining the number of stores each retailer has per state (ranked by the absolute number of store that said retailer has in the state)
Stores_States_Q20_4= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State
FROM Q20_4
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)
Stores_States_Q20_4

,COUNT(Store_Name),Retailer,State
0,5855,Sprouts Farmers Market,CA
1,4310,Whole Foods Market,CA
2,2207,Sprouts Farmers Market,TX
3,2162,Freshmarket,FL
4,1949,Sprouts Farmers Market,AZ
...,...,...,...
119,47,Freshmarket,OK
120,47,EARTHFARE,S
121,47,Sprouts Farmers Market,SC
122,47,Sprouts Farmers Market,VA


### Calculating QoQ growth (4Q19 vs 4Q20)

In [648]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ Traffic growth (in %) data
SortbyID_Q20_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q20_4
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

SortbyID_Q19_4 = sqldf(
"""
    SELECT Store_Name, Retailer, Traffic_N, State,id 
    FROM Q19_4
    GROUP BY (id)
    ORDER BY (id)
    DESC
"""
)

GROW_Q20_4 = sqldf(
"""
    SELECT *
    FROM SortbyID_Q20_4
    INNER JOIN SortbyID_Q19_4 ON [SortbyID_Q20_4].id=SortbyID_Q19_4.id 
"""
)

GROW_Q20_4.columns = ['Store_Name', 'Retailer', 'Traffic_N','State','id','Store_Name_19', 'Retailer_19', 'Traffic_N_19','State_19','id_19']
GROW_Q20_4.drop(['Store_Name_19','Retailer_19', 'State_19', 'id_19'], axis=1, inplace=True)
GROW_Q20_4['Growth'] = 100*((GROW_Q20_4['Traffic_N']/GROW_Q20_4['Traffic_N_19'])-1).mask(GROW_Q20_4.isna(), np.nan)
GROW_Q20_4
GROW_Q20_4.sort_values(['Growth'], ascending=False, inplace=True)
GROW_Q20_4



,Store_Name,Retailer,Traffic_N,State,id,Traffic_N_19,Growth
140,Cave Creek (Store #44),Sprouts Farmers Market,0.417200,AZ,5156149,0.009234,4418.268370
144,Riverview (Store #623),Sprouts Farmers Market,0.343031,FL,5030447,0.009234,3615.020660
20,Wayne,Whole Foods Market,0.482098,N,5610133,0.018467,2510.555058
122,New Tampa (Store #630),Sprouts Farmers Market,0.222507,FL,5586483,0.009234,2309.743131
12,Brookhurst,Whole Foods Market,0.222507,CA,5610141,0.009234,2309.743131
...,...,...,...,...,...,...,...
749,Utah,naturalgrocer,0.009271,UT,2548839,0.083103,-88.843782
286,Woodbury,Whole Foods Market,0.009271,MN,4231666,0.083103,-88.843782
743,Utah,naturalgrocer,0.009271,UT,2548845,0.110804,-91.632836
348,Depaul,Whole Foods Market,0.055627,IL,4231599,0.710990,-92.176159


In [649]:
#Ranking growth data by retailer 
RETAILER_GROW_Q20_4 = sqldf(
"""
    SELECT Retailer, Growth ,
    SUM (Growth) 
    FROM GROW_Q20_4
    GROUP BY (Retailer)
    ORDER BY (Growth)
    DESC
"""
)
RETAILER_GROW_Q20_4.head(6)

,Retailer,Growth,SUM (Growth)
0,Sprouts Farmers Market,4418.268370,19917.918950
1,Whole Foods Market,2510.555058,-7051.242653
2,naturalgrocer,703.261506,-62.926272
3,Freshmarket,151.014909,-4894.334188
4,Freshthyme,108.535463,-687.182428
5,EARTHFARE,30.527753,-517.733396


## Analysing how 2020, specially considering the lockdowns in said year, influenced the QoQ variation in stores that each retailer has.

In [772]:
#Several SQL statements to concated previous and present Quarter data together (using id as the primary key). Ranking QoQ stores growth (in %) data
Stores_States_Q19_4= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State,Traffic_N,id
FROM Q19_4
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)

Stores_States_Q20_4= sqldf(
"""
SELECT COUNT(Store_Name), Retailer, State,Traffic_N,id
FROM Q20_4
GROUP BY State, Retailer
ORDER BY COUNT(Store_Name)
DESC
"""
)

GROW_Stores_Q20_4 = sqldf(
"""
    SELECT *
    FROM Stores_States_Q20_4
    INNER JOIN Stores_States_Q19_4 ON [Stores_States_Q20_4].id=Stores_States_Q19_4.id 
"""
)
GROW_Stores_Q20_4.columns = ['COUNT(Store_Name)', 'Retailer','State','Traffic_N','id','COUNT(Store_Name)_19', 'Retailer_19','State_19','Traffic_N_19','id_19']
GROW_Stores_Q20_4.drop(['Retailer_19', 'State_19', 'id_19'], axis=1, inplace=True)
GROW_Stores_Q20_4['Growth(%)_Stores'] = 100*((GROW_Stores_Q20_4['COUNT(Store_Name)']/GROW_Stores_Q20_4['COUNT(Store_Name)_19'])-1)
GROW_Stores_Q20_4['Growth(%)_Traffic'] = 100*((GROW_Stores_Q20_4['Traffic_N']/GROW_Stores_Q20_4['Traffic_N_19'])-1)
GROW_Stores_Q20_4
GROW_Stores_Q20_4.sort_values(['Growth(%)_Stores'], ascending=False, inplace=True)
GROW_Stores_Q20_4


,COUNT(Store_Name),Retailer,State,Traffic_N,id,COUNT(Store_Name)_19,Traffic_N_19,Growth(%)_Stores,Growth(%)_Traffic
102,94,Sprouts Farmers Market,PA,0.037084,5586490,119,0.009238,-21.008403,301.444774
64,229,Sprouts Farmers Market,MD,0.018542,5586486,364,0.009276,-37.087912,99.886837
106,47,Sprouts Farmers Market,DE,0.185422,5156148,82,0.018467,-42.682927,904.059638
14,1034,Sprouts Farmers Market,FL,0.074169,5610147,1930,0.064635,-46.424870,14.749673
21,684,Whole Foods Market,VA,0.111253,5610137,1277,0.018397,-46.436962,504.740564
...,...,...,...,...,...,...,...,...,...
78,186,naturalgrocer,OK,0.092711,766490,365,0.092336,-49.041096,0.405964
79,185,naturalgrocer,MT,0.074169,766455,364,0.110804,-49.175824,-33.062691
10,1304,Whole Foods Market,IL,0.194694,4973608,2566,0.147738,-49.181606,31.782827
28,548,naturalgrocer,AZ,0.009271,2548844,1083,0.036935,-49.399815,-74.898509


# Demographic analysis

In [823]:
#Loanding the excel file into a pandas df.
dem=pd.read_excel("c:/Users/tiago/OneDrive/Desktop/Python_Notebooks/Advent_business_case/grocers_demographics.xlsx")
dem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   month          1264 non-null   datetime64[ns]
 1   ticker         1264 non-null   object        
 2   id_store       1264 non-null   int64         
 3   income_avg     1264 non-null   int64         
 4   income_25pct   1264 non-null   int64         
 5   income_median  1264 non-null   int64         
 6   income_75pct   1264 non-null   int64         
 7   age            1264 non-null   float64       
 8   male           1264 non-null   float64       
 9   female         1264 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(5), object(1)
memory usage: 98.9+ KB


In [ ]:
#Adjusting date time format
dem['month'] = pd.to_datetime(dem['month']).dt.date

In [ ]:
#Several SQL statements to concated previou Quarter (4Q20) with the avalilable demographic data together (using the time stamp: october-2020 as the primary key to join both datasets).

Stores_States_Q20_4= sqldf(
"""
SELECT *
FROM Q20_4

"""
)

demographics= sqldf(
"""
SELECT *
FROM dem

"""
)
CONCAT_dem = sqldf(
"""
    SELECT *
    FROM demographics
    INNER JOIN Stores_States_Q20_4 ON [demographics].month=Stores_States_Q20_4.day
"""
)


In [837]:
#CONCAT_dem.drop(['month', 'ticker', 'id_store', 'devices_store','devices',], axis=1, inplace=True)
CONCAT_dem


,income_avg,income_25pct,income_median,income_75pct,age,male,female,day,Store_Name,id,Retailer,State,Traffic_N
0,37064,25265,33719,46037,44.787109,0.436599,0.563401,2020-10-01,Arizona,2548844,naturalgrocer,AZ,0.009271
1,37064,25265,33719,46037,44.787109,0.436599,0.563401,2020-10-01,Camarillo (Store #419),3820281,Sprouts Farmers Market,CA,0.009271
2,37064,25265,33719,46037,44.787109,0.436599,0.563401,2020-10-01,Colorado,766403,naturalgrocer,CO,0.009271
3,37064,25265,33719,46037,44.787109,0.436599,0.563401,2020-10-01,Colorado,766429,naturalgrocer,CO,0.009271
4,37064,25265,33719,46037,44.787109,0.436599,0.563401,2020-10-01,Colorado,766481,naturalgrocer,CO,0.009271
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583787,132774,90164,131353,153539,45.392074,0.445170,0.554830,2020-10-01,Utah,5175333,naturalgrocer,UT,0.806587
1583788,132774,90164,131353,153539,45.392074,0.445170,0.554830,2020-10-01,Coral Gables,4231235,Whole Foods Market,FL,1.093992
1583789,132774,90164,131353,153539,45.392074,0.445170,0.554830,2020-10-01,Richard Cooklin,3838593,Freshmarket,VA,1.520464
1583790,132774,90164,131353,153539,45.392074,0.445170,0.554830,2020-10-01,Estero (Store #631),5586491,Sprouts Farmers Market,FL,1.576090


In [842]:
#Ranking stores in each State considering the average income of its visiting costumers
avg_income_grocer= sqldf(
"""
SELECT income_avg, Retailer,State
FROM CONCAT_dem
GROUP BY income_avg, state
ORDER BY income_avg
DESC


"""
)
avg_income_grocer

,income_avg,Retailer,State
0,144316,naturalgrocer,7301
1,144316,Freshmarket,AL
2,144316,naturalgrocer,AR
3,144316,naturalgrocer,AZ
4,144316,Whole Foods Market,C
...,...,...,...
58886,25078,naturalgrocer,UT
58887,25078,Whole Foods Market,VA
58888,25078,naturalgrocer,WA
58889,25078,Freshthyme,WI


In [845]:
#Ranking stores in each State considering the average income of its visiting costumers
age_visitor_grocer= sqldf(
"""
SELECT age, Retailer,State
FROM CONCAT_dem
GROUP BY age,state
ORDER BY age

"""
)
age_visitor_grocer

,age,Retailer,State
0,25.641813,naturalgrocer,7301
1,25.641813,Freshmarket,AL
2,25.641813,naturalgrocer,AR
3,25.641813,naturalgrocer,AZ
4,25.641813,Whole Foods Market,C
...,...,...,...
59403,59.378101,naturalgrocer,UT
59404,59.378101,Whole Foods Market,VA
59405,59.378101,naturalgrocer,WA
59406,59.378101,Freshthyme,WI
